In [13]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

In [5]:
# pip install qiskit-ibm-provider

In [51]:
# pip install pennylane-qulacs["cpu"]

In [52]:
from qiskit_ibm_provider import IBMProvider
# IBMProvider.save_account('')
# from qiskit_ibm_runtime import QiskitRuntimeService

# # Save your credentials on disk.

# service = QiskitRuntimeService(
#     channel='ibm_quantum',
#     instance='ibm-q/open/main',
# )

In [3]:
# symbols, coordinates = qchem.mol_data("BeH2")
# H, qubits = qchem.molecular_hamiltonian(symbols, coordinates)

In [4]:
# H_grouped, qubits_grouped = qchem.molecular_hamiltonian(symbols, coordinates,grouping_type="qwc")
# groups = qml.pauli.group_observables(H.ops, grouping_type='qwc', method='rlf')

In [29]:
# print("Number of terms in ungrouped Hamiltonian: ",len(H.ops) )
# # print("Number of terms in grouped Hamiltonian: ",len(H_grouped.ops) )
# print("Number of terms in grouped Hamiltonian: ",len(groups) )

In [30]:
# print(symbols)
# print(coordinates)
# print("Number of qubits: ", qubits)
# # print("Number of qubits grouped: ", qubits_grouped)
# # print("Qubit Hamiltonian")
# # print(H)

### Hamiltonian without grouping and active space transformation

In [15]:
symbols1 = ["H", "Be", "H"]
coordinates1 = np.array([0.0, 0.0, 2.506532731623098, 0.0, 0.0, 0.0, 0.0, 0.0, -2.506532731623098])

H, qubits = qchem.molecular_hamiltonian(symbols1, coordinates1)
print("Number of qubits: {:}".format(qubits))
print("Qubit Hamiltonian")
H

Number of qubits: 14
Qubit Hamiltonian
<Hamiltonian: terms=666, wires=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]>


### Hamiltonian with grouping and active space transformation

In [16]:
symbols1 = ["H", "Be", "H"]
coordinates1 = np.array([0.0, 0.0, 2.506532731623098, 0.0, 0.0, 0.0, 0.0, 0.0, -2.506532731623098])

H1, qubits1 = qchem.molecular_hamiltonian(symbols1, coordinates1, grouping_type="qwc",grouping_method='rlf', active_electrons=4)
print("Number of qubits: {:}".format(qubits1))
print("Qubit Hamiltonian")
H1

Number of qubits: 12
Qubit Hamiltonian
<Hamiltonian: terms=327, wires=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]>


As can be seen, by using grouping and active space transformation, number of terms and qubits are reduced. 
We will further reduce the the qubits required by tapering the Hamiltonian.

In [17]:
generators = qml.symmetry_generators(H1)
paulixops = qml.paulix_ops(generators, qubits1)

# for idx, generator in enumerate(generators):
    # print(f"generator {idx+1}: {generator}, paulix_op: {paulixops[idx]}")

In [18]:
n_electrons = 4
paulix_sector = qml.qchem.optimal_sector(H1, generators, n_electrons)
print(paulix_sector)

[1, 1, 1, 1, 1]


In [19]:
H_tapered = qml.taper(H1, generators, paulixops, paulix_sector)
# print(H_tapered)
H_tapered

<Hamiltonian: terms=268, wires=[0, 1, 2, 3, 4, 6, 8]>


In [20]:
qubits1 = len(H_tapered.wires)
qubits1

7

In [21]:
state_tapered = qml.qchem.taper_hf(generators, paulixops, paulix_sector,
                                   num_electrons=n_electrons, num_wires=qubits1)
print(state_tapered)

[1 1 1 1 0 0 0]


In [63]:
# dev = qml.device('qiskit.ibmq', wires=H_tapered.wires, backend="ibmq_qasm_simulator", ibmqx_token='83de70e880e6eef552d7924f727414b211092bd4814bb48f57fdf2588fe1cd79c92f2511c2a31c5eb7be586397c88a65aeed06be429493893e0e2b36abf2de77')
dev = qml.device("default.qubit", wires=H_tapered.wires)
# dev = qml.device("qulacs.simulator", wires=H_tapered.wires)

In [64]:
singles, doubles = qml.qchem.excitations(n_electrons, len(H1.wires))
tapered_doubles = [
    qml.taper_operation(qml.DoubleExcitation, generators, paulixops, paulix_sector,
                        wire_order=H1.wires, op_wires=double) for double in doubles
]
tapered_singles = [
    qml.taper_operation(qml.SingleExcitation, generators, paulixops, paulix_sector,
                        wire_order=H1.wires, op_wires=single) for single in singles
]

# dev = qml.device("default.qubit", wires=H_tapered.wires)
@qml.qnode(dev)
def tapered_circuit(params):
    qml.BasisState(state_tapered, wires=H_tapered.wires)
    for idx, tapered_op in enumerate(tapered_doubles + tapered_singles):
        tapered_op(params[idx])
    return qml.expval(H_tapered)

In [65]:
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)
conv_tol = 1e-06
max_iterations = 200
params = np.zeros(len(doubles) + len(singles), requires_grad=True)
energy=[]
for n in range(1, 41):
    params, prev_energy = optimizer.step_and_cost(tapered_circuit, params) 
    # Compute energy
    energy = tapered_circuit(params)

    # Calculate difference between new and old energies
    conv = np.abs(energy - prev_energy)

    if n % 2 == 0:
        print(
            "Iteration = {:},  Energy = {:.8f} Ha,  Convergence parameter = {"
            ":.8f} Ha".format(n, energy, conv)
        )

    if conv <= conv_tol:
        break

print()
print("Final value of the energy = {:.8f} Ha".format(energy))
print("Number of iterations = ", n)

Iteration = 20,  Energy = -15.59446317 Ha,  Convergence parameter = 0.00000154 Ha

Final value of the energy = -15.59446516 Ha
Number of iterations =  22


In [68]:
opt = qml.QNGOptimizer(stepsize=0.2, approx="block-diag")
conv_tol = 1e-06
max_iterations = 200
np.random.seed(79)
params = np.random.rand(len(doubles) + len(singles), requires_grad=True)

qngd_param_history = [params]
qngd_cost_history = []

for n in range(max_iterations):

    # Take step
    params, prev_energy = opt.step_and_cost(tapered_circuit, params)
    qngd_param_history.append(params)
    qngd_cost_history.append(prev_energy)

    # Compute energy
    energy = tapered_circuit(params)

    # Calculate difference between new and old energies
    conv = np.abs(energy - prev_energy)

    if n % 20 == 0:
        print(
            "Iteration = {:},  Energy = {:.8f} Ha,  Convergence parameter = {"
            ":.8f} Ha".format(n, energy, conv)
        )

    if conv <= conv_tol:
        break

print()
print("Final value of the energy = {:.8f} Ha".format(energy))
print("Number of iterations = ", n)

LinAlgError: Singular matrix